<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Deep Learning for NLP
  </div> 
  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
    <font color=orange>I - 3 </font>
  Language Modeling
  </div> 

  <div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 20px; 
      text-align: center; 
      padding: 15px;">
  </div> 

  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Jean-baptiste AUJOGUE
  </div> 

### Part I

1. Word Embedding

2. Sentence Classification

3. <font color=orange>**Language Modeling**</font>

4. Sequence Labelling


### Part II

1. Text Classification

2. Sequence to sequence



### Part III

1. Abstractive Summarization

2. Question Answering

3. Chatbot


</div>

***

<a id="plan"></a>

| | | | |
|------|------|------|------|
| **Content** | [Corpus](#corpus) | [Modules](#modules) | [Model](#model) |

# Overview


Exemples d'implémentation en PyTorch :

- https://github.com/pytorch/examples/blob/master/word_language_model/model.py


Différentes architectures sont décrites dans la litérature :

- Regularizing and Optimizing LSTM Language Models - https://arxiv.org/pdf/1708.02182.pdf

Un modèle linguistique est intérressant en soi, mais peut aussi servir pour le pré-entrainement de couches basses d'un modèle plus complexe :

- Deep contextualized word representations - https://arxiv.org/pdf/1802.05365.pdf
- Improving Language Understanding by Generative Pre-Training - https://www.cs.ubc.ca/~amuham01/LING530/papers/radford2018improving.pdf
- Language Models are Unsupervised Multitask Learners - https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf

# Packages

In [15]:
from __future__ import unicode_literals, print_function, division
import sys
import warnings
import os
from io import open
import unicodedata
import string
import time
import math
import re
import random
import pickle
import copy
from unidecode import unidecode
import itertools
import gc
import multiprocessing

import matplotlib
import matplotlib.pyplot as plt


# for special math operation
from sklearn.preprocessing import normalize


# for manipulating data 
import numpy as np
#np.set_printoptions(threshold=np.nan)
import pandas as pd
import bcolz # see https://bcolz.readthedocs.io/en/latest/intro.html
import pickle


# for text processing
import gensim
from gensim.models import KeyedVectors
#import spacy
import nltk
#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem.porter import PorterStemmer


# for deep learning
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


warnings.filterwarnings("ignore")
print('python version :', sys.version)
print('pytorch version :', torch.__version__)
print('DL device :', device)

python version : 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
pytorch version : 1.4.0
DL device : cuda


In [2]:
path_to_DL4NLP = os.path.dirname(os.getcwd())

In [3]:
sys.path.append(path_to_DL4NLP + '\\lib')

<a id="corpus"></a>

# Corpus

[Back to top](#plan)

Le texte est importé et mis sous forme de liste, où chaque élément représente un texte présenté sous forme d'une liste de mots.

In [4]:
df_GMB_extract = pd.read_csv(path_to_DL4NLP + "\\data\\Groningen Meaning Bank (extract)\\ner.csv", encoding = "ISO-8859-1", error_bad_lines = False)

b'Skipping line 281837: expected 25 fields, saw 34\n'


In [5]:
df_GMB_extract.dropna(inplace = True)
df_GMB_extract = df_GMB_extract[['sentence_idx', 'word', 'pos']]
print(df_GMB_extract.shape)
df_GMB_extract.head()

(1050794, 3)


,sentence_idx,word,pos
0,1.0,Thousands,NNS
1,1.0,of,IN
2,1.0,demonstrators,NNS
3,1.0,have,VBP
4,1.0,marched,VBN


In [6]:
corpus = df_GMB_extract.groupby("sentence_idx").apply(lambda s: [w.lower() for w in s["word"].values.tolist()]).tolist()

In [19]:
len(corpus)

35177

<a id="modules"></a>

# 1 Modules

### 1.1 Word Embedding module

[Back to top](#plan)

We consider here a FastText model trained following the Skip-Gram training objective.

In [7]:
from libDL4NLP.models.Word_Embedding import Word2Vec as myWord2Vec
from libDL4NLP.models.Word_Embedding import Word2VecConnector
from libDL4NLP.utils.Lang import Lang

In [8]:
from gensim.models import Word2Vec
from gensim.test.utils import datapath, get_tmpfile

In [9]:
lang = Lang(corpus, min_count = 1)
print(lang.n_words)

27419


In [16]:
sg_gensim = Word2Vec(corpus, 
                     size = 100, 
                     window = 5, 
                     min_count = 1, 
                     negative = 20, 
                     iter = 25,
                     sg = 1,
                     workers = multiprocessing.cpu_count())

In [10]:
# save
#file_name = get_tmpfile(path_to_DL4NLP + "\\saves\\DL4NLP_I3_skipgram_gensim.model")
#sg_gensim.save(file_name)

# load
#file_name = get_tmpfile(path_to_DL4NLP + "\\saves\\DL4NLP_I3_skipgram_gensim.model")
#sg_gensim = Word2Vec.load(file_name)

In [17]:
word2vec = Word2VecConnector(sg_gensim)

### 1.2 Contextualization module

[Back to top](#plan)

The contextualization layer transforms a sequences of word vectors into another one, of same length, where each output vector corresponds to a new version of each input vector that is contextualized with respect to neighboring vectors.


This module consists of a bi-directional _Gated Recurrent Unit_ (GRU) that supports packed sentences :

In [12]:
from libDL4NLP.modules import RecurrentEncoder

<a id="model"></a>

# 2 Language Model

[Back to top](#plan)


In [20]:
class LanguageModel(nn.Module) :
    def __init__(self, device, tokenizer, word2vec, 
                 hidden_dim = 100, 
                 n_layers = 1, 
                 dropout = 0, 
                 class_weights = None, 
                 optimizer = optim.SGD
                 ):
        super(LanguageModel, self).__init__()
        
        # embedding
        self.tokenizer = tokenizer
        self.word2vec  = word2vec
        self.context   = RecurrentEncoder(self.word2vec.output_dim, hidden_dim, n_layers, dropout, bidirectional = False)
        self.out       = nn.Linear(self.context.output_dim, self.word2vec.lang.n_words)
        self.act       = F.softmax
        
        # optimizer
        self.criterion = nn.NLLLoss(size_average = False, weight = class_weights)
        self.optimizer = optimizer
        
        # load to device
        self.device = device
        self.to(device)
        
    def nbParametres(self) :
        return sum([p.data.nelement() for p in self.parameters() if p.requires_grad == True])
    
    def forward(self, sentence = '.', hidden = None, limit = 10, color_code = '\033[94m'):
        words  = self.tokenizer(sentence)
        result = words + [color_code]
        hidden, count, stop = None, 0, False
        while not stop :
            # compute probs
            embeddings = self.word2vec(words, self.device)
            _, hidden  = self.context(embeddings, lengths = None, hidden = hidden) # WARNING : dim = (n_layers, batch_size, hidden_dim)
            probs      = self.act(self.out(hidden[-1]), dim = 1).view(-1)
            # get predicted word
            topv, topi = probs.data.topk(1)
            words = [self.word2vec.lang.index2word[topi.item()]]
            result += words
            # stopping criterion
            count += 1
            if count == limit or words == [limit] or count == 50 : stop = True
        print(' '.join(result + ['\033[0m']))
        return
    
    def generatePackedSentences(self, sentences, batch_size = 32, depth_range = (5, 10)) :
        sentences = [s[i: i+j] \
                     for s in sentences \
                     for i in range(len(s)-depth_range[0]) \
                     for j in range(depth_range[0], min(depth_range[1], len(s)-i)+1) \
                    ]
        sentences.sort(key = lambda s: len(s), reverse = True)
        packed_data = []
        for i in range(0, len(sentences), batch_size) :
            pack0 = sentences[i:i + batch_size]
            pack0 = [[self.word2vec.lang.getIndex(w) for w in s] for s in pack0]
            pack0 = [[w for w in words if w is not None] for words in pack0]
            pack0.sort(key = len, reverse = True)
            pack1 = Variable(torch.LongTensor([s[-1] for s in pack0]))
            pack0 = [s[:-1] for s in pack0]
            lengths = torch.tensor([len(p) for p in pack0]) # size = (batch_size) 
            pack0 = list(itertools.zip_longest(*pack0, fillvalue = self.word2vec.lang.getIndex('PADDING_WORD')))
            pack0 = Variable(torch.LongTensor(pack0).transpose(0, 1))   # size = (batch_size, max_length) 
            packed_data.append([[pack0, lengths], pack1])
        return packed_data
    
    def fit(self, batches, iters = None, epochs = None, lr = 0.025, random_state = 42,
              print_every = 10, compute_accuracy = True):
        """Performs training over a given dataset and along a specified amount of loops"""
        def asMinutes(s):
            m = math.floor(s / 60)
            s -= m * 60
            return '%dm %ds' % (m, s)

        def timeSince(since, percent):
            now = time.time()
            s = now - since
            rs = s/percent - s
            return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
        
        def computeLogProbs(batch) :
            embeddings = self.word2vec.embedding(batch[0].to(self.device))
            _, hidden  = self.context(embeddings, lengths = batch[1].to(self.device)) # WARNING : dim = (n_layers, batch_size, hidden_dim)
            log_probs  = F.log_softmax(self.out(hidden[-1]), dim = 1)   # dim = (batch_size, lang_size)
            return log_probs

        def computeAccuracy(log_probs, targets) :
            return sum([targets[i].item() == log_probs[i].data.topk(1)[1].item() for i in range(targets.size(0))]) * 100 / targets.size(0)

        def printScores(start, iter, iters, tot_loss, tot_loss_words, print_every, compute_accuracy) :
            avg_loss = tot_loss / print_every
            avg_loss_words = tot_loss_words / print_every
            if compute_accuracy : print(timeSince(start, iter / iters) + ' ({} {}%) loss : {:.3f}  accuracy : {:.1f} %'.format(iter, int(iter / iters * 100), avg_loss, avg_loss_words))
            else                : print(timeSince(start, iter / iters) + ' ({} {}%) loss : {:.3f}                     '.format(iter, int(iter / iters * 100), avg_loss))
            return 0, 0

        def trainLoop(batch, optimizer, compute_accuracy = True):
            """Performs a training loop, with forward pass, backward pass and weight update."""
            torch.cuda.empty_cache()
            optimizer.zero_grad()
            self.zero_grad()
            log_probs = computeLogProbs(batch[0])
            targets   = batch[1].to(self.device).view(-1)
            loss      = self.criterion(log_probs, targets)
            loss.backward()
            optimizer.step() 
            accuracy = computeAccuracy(log_probs, targets) if compute_accuracy else 0
            return float(loss.item() / targets.size(0)), accuracy
        
        # --- main ---
        self.train()
        np.random.seed(random_state)
        start = time.time()
        optimizer = self.optimizer([param for param in self.parameters() if param.requires_grad == True], lr = lr)
        tot_loss = 0  
        tot_acc  = 0
        if epochs is None :
            for iter in range(1, iters + 1):
                batch = random.choice(batches)
                loss, acc = trainLoop(batch, optimizer, compute_accuracy)
                tot_loss += loss
                tot_acc += acc      
                if iter % print_every == 0 : 
                    tot_loss, tot_acc = printScores(start, iter, iters, tot_loss, tot_acc, print_every, compute_accuracy)
        else :
            iter = 0
            iters = len(batches) * epochs
            for epoch in range(1, epochs + 1):
                print('epoch ' + str(epoch))
                np.random.shuffle(batches)
                for batch in batches :
                    loss, acc = trainLoop(batch, optimizer, compute_accuracy)
                    tot_loss += loss
                    tot_acc += acc 
                    iter += 1
                    if iter % print_every == 0 : 
                        tot_loss, tot_acc = printScores(start, iter, iters, tot_loss, tot_acc, print_every, compute_accuracy)
        return

### Training

In [18]:
language_model = LanguageModel(device,
                               tokenizer = lambda s : s.split(' '),
                               word2vec = word2vec,
                               hidden_dim = 100, 
                               n_layers = 3, 
                               dropout = 0.1,
                               optimizer = optim.AdamW)

language_model.nbParametres()

2951220

In [22]:
batches  = language_model.generatePackedSentences(corpus, batch_size = 64, depth_range = (5, 6))
batches += language_model.generatePackedSentences(corpus, batch_size = 64, depth_range = (10, 11))
len(batches)

50335

In [24]:
language_model.fit(batches, epochs = 1, lr = 0.001, print_every = 250)

epoch 1
0m 18s (- 61m 52s) (250 0%) loss : 7.775  accuracy : 5.0 %
0m 36s (- 60m 50s) (500 0%) loss : 7.346  accuracy : 5.3 %
0m 54s (- 60m 28s) (750 1%) loss : 7.218  accuracy : 5.7 %
1m 12s (- 59m 59s) (1000 1%) loss : 7.093  accuracy : 7.2 %
1m 31s (- 59m 35s) (1250 2%) loss : 7.010  accuracy : 8.6 %
1m 49s (- 59m 11s) (1500 2%) loss : 6.857  accuracy : 8.8 %
2m 7s (- 58m 52s) (1750 3%) loss : 6.840  accuracy : 9.0 %
2m 25s (- 58m 31s) (2000 3%) loss : 6.683  accuracy : 10.0 %
2m 43s (- 58m 11s) (2250 4%) loss : 6.721  accuracy : 10.3 %
3m 1s (- 57m 52s) (2500 4%) loss : 6.551  accuracy : 11.2 %
3m 19s (- 57m 31s) (2750 5%) loss : 6.508  accuracy : 11.6 %
3m 37s (- 57m 11s) (3000 5%) loss : 6.417  accuracy : 12.3 %
3m 55s (- 56m 53s) (3250 6%) loss : 6.394  accuracy : 12.6 %
4m 13s (- 56m 33s) (3500 6%) loss : 6.373  accuracy : 12.4 %
4m 32s (- 56m 20s) (3750 7%) loss : 6.344  accuracy : 12.4 %
4m 50s (- 56m 1s) (4000 7%) loss : 6.240  accuracy : 13.8 %
5m 8s (- 55m 44s) (4250 8%) l

KeyboardInterrupt: 

In [34]:
batches  = language_model.generatePackedSentences(corpus, batch_size = 64, depth_range = (5, 6))
batches += language_model.generatePackedSentences(corpus, batch_size = 64, depth_range = (10, 11))
batches += language_model.generatePackedSentences(corpus, batch_size = 64, depth_range = (15, 16))
len(batches)

66148

In [62]:
language_model.fit(batches, epochs = 1, lr = 0.00025, print_every = 250)

epoch 1
0m 18s (- 12m 5s) (250 2%) loss : 5.280  accuracy : 19.5 %
0m 37s (- 11m 43s) (500 5%) loss : 5.279  accuracy : 19.6 %
0m 55s (- 11m 23s) (750 7%) loss : 5.280  accuracy : 19.9 %
1m 13s (- 11m 4s) (1000 10%) loss : 5.421  accuracy : 19.0 %
1m 32s (- 10m 46s) (1250 12%) loss : 5.371  accuracy : 18.8 %
1m 50s (- 10m 27s) (1500 15%) loss : 5.316  accuracy : 19.5 %
2m 9s (- 10m 8s) (1750 17%) loss : 5.304  accuracy : 19.4 %
2m 27s (- 9m 49s) (2000 20%) loss : 5.464  accuracy : 18.9 %
2m 45s (- 9m 31s) (2250 22%) loss : 5.406  accuracy : 19.6 %
3m 4s (- 9m 12s) (2500 25%) loss : 5.380  accuracy : 19.6 %
3m 22s (- 8m 54s) (2750 27%) loss : 5.376  accuracy : 19.2 %
3m 40s (- 8m 35s) (3000 30%) loss : 5.310  accuracy : 19.4 %
3m 59s (- 8m 16s) (3250 32%) loss : 5.287  accuracy : 20.4 %
4m 17s (- 7m 58s) (3500 35%) loss : 5.327  accuracy : 19.2 %
4m 35s (- 7m 39s) (3750 37%) loss : 5.308  accuracy : 19.6 %
4m 53s (- 7m 20s) (4000 40%) loss : 5.332  accuracy : 19.2 %
5m 11s (- 7m 2s) (42

KeyboardInterrupt: 

In [74]:
# save
#torch.save(language_model.state_dict(), path_to_DL4NLP + '\\saves\\DL4NLP_I3_language_model.pth')

# load
#language_model.load_state_dict(torch.load(path_to_DL4NLP + '\\saves\\DL4NLP_I3_language_model.pth'))

#### Evaluation

In [73]:
# fastText gensim, n_layers = 3, dh = 50
language_model.eval()
sentence = random.choice(corpus)
i = random.choice(range(int(len(sentence)/2)))
sentence = ' '.join(sentence[:i]) if i > 0 else '.'
language_model(sentence, limit = '.', color_code = '\x1b[48;2;255;229;217m') #  '\x1b[48;2;255;229;217m' '\x1b[31m'

at least 6,000  people were killed in the attack . 
